# Create Agents to Research and Write an Article

Introducing to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.


```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI  # Updated import statement

- As a LLM agents, I used `ollama llama3.2` .

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
# Set environment variables for API and LLM base URL
os.environ["OPENAI_API_KEY"] = ""  # Insert your OpenAI API key here
os.environ["LLM_BASE_URL"] = "http://localhost:11434"  # Local LLM endpoint
os.environ["OTEL_SDK_DISABLED"] = "true"

# Initialize the language model from OpenAI with the specified base URL and model
llm = ChatOpenAI(
    model="ollama/llama3.2",  # Example model; replace with your desired Ollama model
    base_url=os.environ["LLM_BASE_URL"]
)

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic} in 'https://medium.com/'."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "You have to prepare a detailed "
              "outline and the relevant topics and sub-topics that has to be a part of the"
              "blogpost."
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    llm=llm,
    allow_delegation=False,
 verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic} in 'https://medium.com/'. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization 'https://medium.com/'. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="Very short content plan document "
        "with a short outline, audience analysis, "
        "a few SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
  "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

In [11]:
inputs = {"topic":"Comparative study of LangGraph, Autogen and Crewai for building multi-agent system."}
result = crew.kickoff(inputs=inputs)

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Comparative study of LangGraph, Autogen and Crewai for building multi-agent system..
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
**Comparative Study of LangGraph, Autogen, and Crewai for Building Multi-Agent Systems**

**Introduction:**
The increasing demand for intelligent systems that can interact with each other has led to the development of multi-agent systems. In this article, we will compare LangGraph, Autogen, and Crewai, three popular platforms used for building multi-agent systems.

**Audience Analysis:**
Our target audience consists of:

* Developers interested in building multi-agent systems
* Researchers studying multi-agent systems
* Individual

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result.raw)

Comparative Study of LangGraph, Autogen, and Crewai for Building Multi-Agent Systems
=====================================================================================

### Introduction

The increasing demand for intelligent systems that can interact with each other has led to the development of multi-agent systems. In this article, we will compare LangGraph, Autogen, and Crewai, three popular platforms used for building multi-agent systems.

As the field of artificial intelligence (AI) continues to evolve, multi-agent systems are becoming increasingly important in various industries such as robotics, finance, and healthcare. However, building complex multi-agent systems can be a daunting task, requiring expertise in multiple areas including AI, computer science, and engineering.

In this article, we will delve into the world of LangGraph, Autogen, and Crewai, three platforms that are revolutionizing the way we build multi-agent systems. From their strengths and weaknesses to their use cases and applications, we will explore what each platform has to offer and how they can help you achieve your goals.

### Audience Analysis

Our target audience consists of:

* Developers interested in building multi-agent systems
* Researchers studying multi-agent systems
* Individuals looking to adopt AI-based solutions

Whether you are a seasoned developer or a researcher looking for a new tool, this article is designed to provide you with the information and insights you need to make informed decisions about which platform to use.

### Key Points

#### Overview of LangGraph

LangGraph is an open-source framework for building multi-agent systems. It provides a simple and intuitive API for creating agents and interactions between them. LangGraph is highly customizable and can be used with various programming languages.

One of the strengths of LangGraph is its flexibility, allowing developers to create complex systems from scratch. However, this also means that it requires advanced programming skills, which may be a barrier for some users.

#### Overview of Autogen

Autogen is a cloud-based platform for building multi-agent systems. It provides a visual interface for designing agents and their interactions, making it easy to use for non-technical users. Autogen offers a range of pre-built templates and plugins for common use cases.

The ease of use offered by Autogen makes it an attractive option for researchers or developers who want to build a multi-agent system quickly. However, its customizability is limited compared to LangGraph.

#### Overview of Crewai

Crewai is an AI-powered platform for building multi-agent systems. It provides a set of pre-trained models and algorithms for creating agents that can learn and adapt to new situations. Crewai offers real-time monitoring and analytics for optimizing agent performance.

The use of pre-trained models in Crewai makes it an attractive option for researchers who want to build complex AI-powered systems quickly. However, its reliance on AI power means that it may not be suitable for all applications.

### Comparison of LangGraph, Autogen, and Crewai

| Feature | LangGraph | Autogen | Crewai |
| --- | --- | --- | --- |
| Open-source | Yes | No | No |
| Customizability | High | Medium | Low |
| Ease of use | Advanced | Beginner-friendly | Intermediate |

#### Comparison Points

* **Customizability**: LangGraph offers high customizability, making it suitable for complex systems. Autogen provides medium customizability, while Crewai offers low customizability due to its reliance on pre-trained models.
* **Ease of use**: Autogen provides a beginner-friendly interface, making it easy to use for non-technical users. LangGraph requires advanced programming skills, while Crewai falls in between.

### Conclusion

In conclusion, each platform has its strengths and weaknesses. LangGraph offers high customizability but requires advanced programming skills. Autogen provides a visual interface for easy use but is limited in terms of customization. Crewai provides pre-trained models and algorithms but relies on AI power, which can be unpredictable.

For developers interested in building multi-agent systems, we recommend starting with LangGraph or Autogen based on their specific needs and skill level. For researchers looking for a plug-and-play solution, Crewai might be the best option.

### Call to Action

* For developers: Start with LangGraph or Autogen based on your skills and project requirements.
* For researchers: Consider using Crewai for its pre-trained models and algorithms.

### Resources

* LangGraph: <https://langgraph.org/>
* Autogen: <https://autogen.ai/>
* Crewai: <https://crew.ai/>

### SEO Keywords

* Multi-agent systems
* LangGraph
* Autogen
* Crewai
* AI-powered platforms
* Cloud-based solutions